# 9.1题求解过程

In [1]:
import chapter9module as c9m

In [2]:
# 输入变量的值
a = 2
b = 0.6
t = 0.012
q = 6.5 / 0.01 / 0.01
sigma_0 = 18.8e6

求解（a）：
1. 计算A点弯矩：

In [7]:
M_A = c9m.bendingMomentFixedEnd1(q, b)
print(f"A点弯矩 M_A 为 {M_A :.2f} N*m\n")

A点弯矩 M_A 为 975.00 N*m



In [9]:
sigma_A = c9m.bendingNormalStress(M_A, t)
print(f"A点由弯曲引起的正应力 sigma_A 为 {sigma_A*1e-6 :.2f} MPa\n")

A点由弯曲引起的正应力 sigma_A 为 40.62 MPa

